<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/data_connectors/simple_directory_reader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# 一个简单的远程文件系统目录读取器

这个Python脚本用于从远程文件系统中读取目录的内容。


`SimpleDirectoryReader`是最常用的数据连接器，它可以直接使用。默认情况下，它可以用来解析本地文件系统中的各种文件类型，并将其转换为`Document`对象的列表。此外，它还可以轻松配置为从远程文件系统中读取数据！这是通过[`fsspec`](https://filesystem-spec.readthedocs.io/en/latest/index.html)协议实现的。

本笔记本将带您通过一个示例，演示如何使用`SimpleDirectoryReader`从S3存储桶加载文档。您可以将其运行在实际的S3存储桶上，或者在通过[LocalStack](https://www.localstack.cloud/)模拟的本地S3存储桶上。


### 开始使用


如果您在colab上打开此笔记本，您可能需要安装LlamaIndex 🦙。


In [ ]:
!pip install llama-index s3fs boto3


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


下载数据


In [ ]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay1.txt'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay2.txt'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay3.txt'

--2024-03-21 16:03:10--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8000::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-03-21 16:03:11 ERROR 404: Not Found.

--2024-03-21 16:03:11--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8000::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-03-21 16:03:11 ERROR 404: Not Found.

--2024-03-21 16:03:11--  https://raw

In [ ]:
# 在S3中创建一个测试桶import boto3endpoint_url = (    "http://localhost:4566"  # 如果您正在使用localstack访问S3，请使用此行)# endpoint_url = None  # 如果您正在使用真实的AWS S3，请使用此行bucket_name = "llama-index-test-bucket"s3 = boto3.resource("s3", endpoint_url=endpoint_url)s3.create_bucket(Bucket=bucket_name)bucket = s3.Bucket(bucket_name)# 将paul graham的文章放入测试桶中的不同子目录bucket.upload_file(    "data/paul_graham/paul_graham_essay1.txt", "essays/paul_graham_essay1.txt")bucket.upload_file(    "data/paul_graham/paul_graham_essay2.txt",    "essays/more_essays/paul_graham_essay2.txt",)bucket.upload_file(    "data/paul_graham/paul_graham_essay3.txt",    "essays/even_more_essays/paul_graham_essay3.txt",)

In [ ]:
from llama_index.core import SimpleDirectoryReader

In [ ]:
# 使用s3fs创建文件系统from s3fs import S3FileSystems3_fs = S3FileSystem(anon=False, endpoint_url=endpoint_url)

加载特定文件


In [ ]:
reader = SimpleDirectoryReader(    input_dir=bucket_name,    fs=s3_fs,    recursive=True,  # 递归搜索所有子目录)

In [ ]:
docs = reader.load_data()
print(f"Loaded {len(docs)} docs")

Loaded 3 docs


加载目录中的所有（顶层）文件


In [ ]:
reader = SimpleDirectoryReader(input_dir="./data/paul_graham/")

In [ ]:
docs = reader.load_data()print(f"加载了 {len(docs)} 个文档")# 显示每个文档的元数据for idx, doc in enumerate(docs):    print(f"{idx} - {doc.metadata}")

Loaded 3 docs
0 - {'file_path': '/Users/sourabhdesai/workspace/llama_index/docs/docs/examples/data_connectors/data/paul_graham/paul_graham_essay1.txt', 'file_name': 'paul_graham_essay1.txt', 'file_type': 'text/plain', 'file_size': 0, 'creation_date': '2024-03-21', 'last_modified_date': '2024-03-21'}
1 - {'file_path': '/Users/sourabhdesai/workspace/llama_index/docs/docs/examples/data_connectors/data/paul_graham/paul_graham_essay2.txt', 'file_name': 'paul_graham_essay2.txt', 'file_type': 'text/plain', 'file_size': 0, 'creation_date': '2024-03-21', 'last_modified_date': '2024-03-21'}
2 - {'file_path': '/Users/sourabhdesai/workspace/llama_index/docs/docs/examples/data_connectors/data/paul_graham/paul_graham_essay3.txt', 'file_name': 'paul_graham_essay3.txt', 'file_type': 'text/plain', 'file_size': 0, 'creation_date': '2024-03-21', 'last_modified_date': '2024-03-21'}


In [ ]:
docs = reader.load_data()print(f"加载了 {len(docs)} 个文档")# 显示每个文档的元数据for idx, doc in enumerate(docs):    print(f"{idx} - {doc.metadata}")

Loaded 3 docs
0 - {'file_path': '/Users/sourabhdesai/workspace/llama_index/docs/docs/examples/data_connectors/data/paul_graham/paul_graham_essay1.txt', 'file_name': 'paul_graham_essay1.txt', 'file_type': 'text/plain', 'file_size': 0, 'creation_date': '2024-03-21', 'last_modified_date': '2024-03-21'}
1 - {'file_path': '/Users/sourabhdesai/workspace/llama_index/docs/docs/examples/data_connectors/data/paul_graham/paul_graham_essay2.txt', 'file_name': 'paul_graham_essay2.txt', 'file_type': 'text/plain', 'file_size': 0, 'creation_date': '2024-03-21', 'last_modified_date': '2024-03-21'}
2 - {'file_path': '/Users/sourabhdesai/workspace/llama_index/docs/docs/examples/data_connectors/data/paul_graham/paul_graham_essay3.txt', 'file_name': 'paul_graham_essay3.txt', 'file_type': 'text/plain', 'file_size': 0, 'creation_date': '2024-03-21', 'last_modified_date': '2024-03-21'}


创建一个迭代器来加载文件并在加载时处理它们


In [ ]:
reader = SimpleDirectoryReader(    input_dir=bucket_name,    fs=s3_fs,    recursive=True,)all_docs = []for docs in reader.iter_data():    for doc in docs:        # 对文档进行处理        doc.text = doc.text.upper()        all_docs.append(doc)print(len(all_docs))

3


在远程文件系统上排除特定模式


In [ ]:
读取器 = SimpleDirectoryReader(    input_dir=bucket_name,  # 输入目录为bucket_name    fs=s3_fs,  # 文件系统为s3_fs    recursive=True,  # 递归为True    exclude=["essays/more_essays/*"],  # 排除["essays/more_essays/*"])all_docs = []for docs in reader.iter_data():  # 对于读取器中的数据进行迭代    for doc in docs:  # 对于每个文档进行迭代        # 对文档进行处理        doc.text = doc.text.upper()  # 将文档文本转换为大写        all_docs.append(doc)  # 将文档添加到all_docs列表中print(len(all_docs))  # 打印all_docs列表的长度all_docs  # 打印all_docs列表

2


[Document(id_='8329e3e6-a16e-4aca-8a85-3b338d888925', embedding=None, metadata={'file_path': 'llama-index-test-bucket/essays/even_more_essays/paul_graham_essay3.txt', 'file_name': 'paul_graham_essay3.txt', 'file_type': 'text/plain', 'file_size': 0}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='cc681e3c-47f6-4e34-8e8b-e2bb0b44ea2d', embedding=None, metadata={'file_path': 'llama-index-test-bucket/essays/paul_graham_essay1.txt', 'file_name': 'paul_graham_essay1.txt', 'file_type': 'text/plain', 'file_size': 0}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date

异步执行可通过`aload_data`实现


In [ ]:
import nest_asyncio

nest_asyncio.apply()

reader = SimpleDirectoryReader(
    input_dir=bucket_name,
    fs=s3_fs,
    recursive=True,
)

all_docs = await reader.aload_data()

print(len(all_docs))

3
